In [69]:
import notebook_setup

import jax.numpy as jnp
import pandas as pd

from faker import Faker
from src.entities.shifts import Shift
from src.entities.agents import Agent

In [70]:
# Data Setup

faker = Faker()

agent_emails = [faker.email() for _ in range(200)]

skills = ["chat_pt", "chat_es", "whatchmen"]

shifts = [
    *[
        Shift(
            id=f"{'chat_pt'}_{start}_({9.0})",
            skill="chat_pt",
            start_time=start,
            duration=9,
        )
        for start in jnp.arange(0.0, 24.0, 1.0)
    ],
    *[
        Shift(
            id=f"{'chat_es'}_{start}_({8.5})",
            skill="chat_es",
            start_time=start,
            duration=8.5,
        )
        for start in jnp.arange(0, 24, 6)
    ],
    *[
        Shift(
            id=f"{'whatchmen'}_{start}_({9.0})",
            skill="whatchmen",
            start_time=start,
            duration=9,
        )
        for start in [8.0, 16.0]
    ],
]

datetime.date(2024, 8, 12)

[Shift(id='chat_pt_0.0_(9.0)', start_time=Array(0., dtype=float32), duration=9, skill='chat_pt'),
 Shift(id='chat_pt_1.0_(9.0)', start_time=Array(1., dtype=float32), duration=9, skill='chat_pt'),
 Shift(id='chat_pt_2.0_(9.0)', start_time=Array(2., dtype=float32), duration=9, skill='chat_pt'),
 Shift(id='chat_pt_3.0_(9.0)', start_time=Array(3., dtype=float32), duration=9, skill='chat_pt'),
 Shift(id='chat_pt_4.0_(9.0)', start_time=Array(4., dtype=float32), duration=9, skill='chat_pt'),
 Shift(id='chat_pt_5.0_(9.0)', start_time=Array(5., dtype=float32), duration=9, skill='chat_pt'),
 Shift(id='chat_pt_6.0_(9.0)', start_time=Array(6., dtype=float32), duration=9, skill='chat_pt'),
 Shift(id='chat_pt_7.0_(9.0)', start_time=Array(7., dtype=float32), duration=9, skill='chat_pt'),
 Shift(id='chat_pt_8.0_(9.0)', start_time=Array(8., dtype=float32), duration=9, skill='chat_pt'),
 Shift(id='chat_pt_9.0_(9.0)', start_time=Array(9., dtype=float32), duration=9, skill='chat_pt'),
 Shift(id='chat_pt_1

In [71]:
agent_emails


schedules = {}

availability = {}

In [74]:
team1 = agent_emails[0:10]

In [75]:
team1

['henrymichelle@example.net',
 'cynthia45@example.org',
 'tanyaallen@example.org',
 'kimberlyrobinson@example.org',
 'greenmartha@example.com',
 'joseph61@example.com',
 'fitzgeraldwilliam@example.net',
 'ashleymcknight@example.net',
 'grantheather@example.com',
 'jill65@example.org']